In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 15
WINTER = "2024-2025"
TITLE = "4-5 March 2025"
SUBTITLE = "8 AM 6 March 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 3, 5, 5)
ets = datetime(2025, 3, 18, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,MI,MQT,24.0,-87.580000,46.590000,POINT (944435.526 249965.113),True,0,True,LSR,22,26
1,MI,MQT,23.0,-88.330000,46.650000,POINT (886945.54 248019.455),True,1,True,LSR,20,26
2,MI,MQT,22.7,-87.550000,46.530000,POINT (947737.524 243709.461),True,2,True,LSR,22,26
3,MI,MQT,22.0,-87.610000,46.560000,POINT (942696.48 246307.558),True,3,True,LSR,22,26
4,MI,MQT,21.0,-88.670000,47.040000,POINT (855125.211 287340.693),True,4,True,LSR,19,27
...,...,...,...,...,...,...,...,...,...,...,...,...
3759,NE,OAX,0.0,-96.615801,40.824946,POINT (284844.376 -458300.283),False,NE-LA-82,False,COCORAHS,0,2
3760,NE,OAX,0.0,-96.609223,40.768958,POINT (285643.634 -464495.353),False,NE-LA-86,False,COCORAHS,0,2
3761,NE,OAX,0.0,-96.705310,40.779730,POINT (277508.578 -463626.866),False,NE-LA-88,False,COCORAHS,0,2
3762,NE,OAX,0.0,-95.966330,40.680430,POINT (340214.752 -471888.476),False,NE-OT-5,False,COCORAHS,2,2


In [31]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'IA-CR-7',
        'MSNI4',
        1101,
        'WSHI4',
        'BNWI4',
        'IA-BN-3',
        'IA-CY-6',
        'SDHI4',
        441,
        704,
        'IA-ML-5',
        'GRII4',
        'LOGI4',
        'IA-HR-1',
        'NE-SY-46',
        'IA-CT-11',
        'MTAI4',
        'OSEI4',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('AUDI4', 4),
        (630, 4),
        ('IA-CF-2', 3),
        ('IA-JN-10', 1.4),
        ('WTRI4', 3.6),
        (790, 2.5),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=0.5)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250306.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
150     IA  DMX  9.0 -93.940000  43.390000  POINT (489163.688 -160837.545)   
308     IA  DMX  6.3 -94.640000  41.500000  POINT (446196.816 -374578.588)   
321     IA  DMX  6.0 -95.010000  41.400000  POINT (416097.083 -387616.784)   
353     IA  DMX  5.5 -94.630000  42.400000  POINT (440664.607 -274634.939)   
354     IA  DMX  5.5 -94.500000  41.680000  POINT (456532.974 -353834.054)   
...    ...  ...  ...        ...        ...                             ...   
3755    IA  OAX  0.0 -95.386238  41.752669   POINT (382635.184 -350293.23)   
3756    IA  OAX  0.0 -95.698426  40.724626  POINT (362540.206 -465841.464)   
3757    IA  FSD  0.0 -96.332469  43.054526  POINT (297907.322 -209629.327)   
3758    IA  FSD  0.0 -96.055731  43.008573  POINT (320607.488 -213686.931)   
3763    IA  FSD  0.0 -95.142910  43.044500  POINT (394473.817 -205701.937)   

      used_for_analysis     nwsli  plotme    source  xcell  yce

## 